In [ ]:
# This code is for the first project I did at Metis, which was an Exploratory Data Analysis project for MTA data
# This was a group project, so I do not have a local copy of most of the code we used for the project
# However, here is some code that we used at the start to aggregate multiple weeks of data from the MTA website
# It has been modified to allow a user to start at any date and request any number of weeks of data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import date
import time

In [ ]:
# This function takes a start date and number of weeks as arguments
# The format is get_MTA_turnstile_data("YYYYMMDD", Integer), for example "20190301" is March 1st, 2019

# Note that this will only work for start dates October 12, 2014 (20141012) or later
# Data from prior to that have a different layout on the MTA website

# Note also that you may get a few days of data prior to your start date, since the data are broken up by weeks

def get_MTA_turnstile_data(Start_Date, Number_Of_Weeks):
    
    Today = str(date.today())
    Current_Year_Int = int(Today[0:4])
    Current_Year_Int_Last_Two = int(Today[2:4])
    Current_Month_Int = int(Today[5:7])
    Current_Day_Int = int(Today[8:10])
    
    Start_Date_String = str(Start_Date)
    
    if(len(Start_Date_String) != 8):
        return "Your input must be a date string in the format 'YYYYMMDD'"
    
    Full_Year_String = Start_Date_String[0:4]
    Full_Year_Int = int(Full_Year_String)
    
    Year_String = Start_Date_String[2:4]
    Month_String = Start_Date_String[4:6]
    Day_String = Start_Date_String[6:8]
    
    Full_Date_String = Year_String + Month_String + Day_String
    
    Year_Int = int(Year_String)
    Month_Int = int(Month_String)
    Day_Int = int(Day_String)
    
    Normal_Month_Days = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    Leap_Year_Month_Days = [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    Leap_Year = False
    
    if((Year_Int % 4) == 0):
        if((Year_Int % 100) == 0):
            if((Year_Int % 400) == 0):
                Leap_Year = True
        else:
            Leap_Year = True
            
    if((Month_Int > 12) or (Month_Int < 1)):
        return "Invalid Date"
    
    if(Day_Int < 1):
        return "Invalid Date"
            
    if(Leap_Year == True):
        if(Day_Int > Leap_Year_Month_Days[Month_Int]):
            return "Invalid Date"
    else:
        if(Day_Int > Normal_Month_Days[Month_Int]):
            return "Invalid Date"
    
    if((Full_Year_Int < 2014) or (Full_Year_Int > Current_Year_Int)):
        if(Full_Year_Int < 2014):
            return "Please select a date on or after October 12, 2014"
        else:
            return "Start date can not be a date in the future"
    elif(Full_Year_Int == 2014):
        if(Month_Int < 10):
            return "Please select a date on or after October 12, 2014"
        elif(Month_Int == 10):
            if(Day_Int < 12):
                return "Please select a date on or after October 12, 2014"
    elif(Full_Year_Int == Current_Year_Int):
        if(Month_Int > Current_Month_Int):
            return "Start date can not be a date in the future"
        elif(Month_Int == Current_Month_Int):
            if(Day_Int > Current_Day_Int):
                return "Start date can not be a date in the future"
            
    Records_Found = False
    Current_Day_Reached = False
    Weeks_Of_Records_Added = 0
    
    Start_Of_URL = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_'
    End_Of_URL = '.txt'
    
    while((Records_Found == False) and (Current_Day_Reached == False)):
        
        URL_String = Start_Of_URL + Full_Date_String + End_Of_URL
        
        try:
            MTA_Turnstile_DF = pd.read_csv(URL_String)
            Records_Found = True
            Weeks_Of_Records_Added = 1
            
        except:

            Day_Int += 1
            New_Month = False

            if(Leap_Year == True):
                if(Day_Int > Leap_Year_Month_Days[Month_Int]):
                    Day_Int -= Leap_Year_Month_Days[Month_Int]
                    New_Month = True
            else:
                if(Day_Int > Normal_Month_Days[Month_Int]):
                    Day_Int -= Normal_Month_Days[Month_Int]
                    New_Month = True
                    
            if(New_Month == True):
                if(Month_Int < 12):            
                    Month_Int += 1            
                else:
                    Month_Int = 1
                    Year_Int += 1
                    
                    if((Year_Int % 4) == 0):
                        if((Year_Int % 100) == 0):
                            if((Year_Int % 400) == 0):
                                Leap_Year = True
                            else:
                                Leap_Year = False
                        else:
                            Leap_Year = True
                    else:
                        Leap_Year = False

            Year_String = str(Year_Int)
            
            if(Month_Int > 9):
                Month_String = str(Month_Int)
            else:
                Month_String = "0" + str(Month_Int)
                
            if(Day_Int > 9):
                Day_String = str(Day_Int)
            else:
                Day_String = "0" + str(Day_Int)

            Full_Date_String = Year_String + Month_String + Day_String

            if(Year_Int > Current_Year_Int_Last_Two):
                Current_Day_Reached = True
            elif(Year_Int == Current_Year_Int_Last_Two):
                if(Month_Int > Current_Month_Int):
                    Current_Day_Reached = True
                elif(Month_Int == Current_Month_Int):
                    if(Day_Int >= Current_Day_Int):
                        Current_Day_Reached = True
                
    if(Records_Found == False):
        return "No Records Found, try a date more than one week ago"
    
    while((Weeks_Of_Records_Added < Number_Of_Weeks) and (Current_Day_Reached == False)):
        
        Day_Int += 7
        New_Month = False

        if(Leap_Year == True):
            if(Day_Int > Leap_Year_Month_Days[Month_Int]):
                Day_Int -= Leap_Year_Month_Days[Month_Int]
                New_Month = True
        else:
            if(Day_Int > Normal_Month_Days[Month_Int]):
                Day_Int -= Normal_Month_Days[Month_Int]
                New_Month = True

        if(New_Month == True):
            if(Month_Int < 12):            
                Month_Int += 1            
            else:
                Month_Int = 1
                Year_Int += 1

                if((Year_Int % 4) == 0):
                    if((Year_Int % 100) == 0):
                        if((Year_Int % 400) == 0):
                            Leap_Year = True
                        else:
                            Leap_Year = False
                    else:
                        Leap_Year = True
                else:
                    Leap_Year = False

        Year_String = str(Year_Int)
        
        if(Month_Int > 9):
            Month_String = str(Month_Int)
        else:
            Month_String = "0" + str(Month_Int)
            
        if(Day_Int > 9):
            Day_String = str(Day_Int)
        else:
            Day_String = "0" + str(Day_Int)

        Full_Date_String = Year_String + Month_String + Day_String
        URL_String = Start_Of_URL + Full_Date_String + End_Of_URL
        
        try:
            New_Data_Frame = pd.read_csv(URL_String)
            MTA_Turnstile_DF = MTA_Turnstile_DF.append(New_Data_Frame, ignore_index = True)
            Weeks_Of_Records_Added += 1
            
        except:
            print("No records found for week before", ("20" + Full_Date_String))
        
        if(Year_Int > Current_Year_Int_Last_Two):
            Current_Day_Reached = True
        elif(Year_Int == Current_Year_Int_Last_Two):
            if(Month_Int > Current_Month_Int):
                Current_Day_Reached = True
            elif(Month_Int == Current_Month_Int):
                if(Day_Int >= Current_Day_Int):
                    Current_Day_Reached = True
                    
    if(Weeks_Of_Records_Added < Number_Of_Weeks):
        print("We could only find", Weeks_Of_Records_Added, "weeks of records between your start date and today")
        
    if(Records_Found == True):
        # The line below automatically takes the white space out of the top of the 'EXITS' column
        MTA_Turnstile_DF.rename(columns={'EXITS                                                               ': 'EXITS'}, inplace=True)
        return MTA_Turnstile_DF

In [ ]:
# Here is an example that pulls twelve weeks of records, starting from the week of December 20, 2019
# May take a few minutes to run

Example_DF = get_MTA_turnstile_data("20191220", 12)
display(Example_DF)

In [ ]:
# You can get your own MTA turnstile records by filling in your preferred start date and number of weeks
# De-commentify the comment below and run to get results

#My_MTA_DF = get_MTA_turnstile_data(Start_Date, Number_Of_Weeks)
#display(My_MTA_DF)